<a href="https://colab.research.google.com/github/Sherin1998/hotel-attrition/blob/main/Task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imposting necessary libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline 
from pandas_profiling import ProfileReport
from sklearn.impute import SimpleImputer
pd.set_option('display.max_columns', None)

In [ ]:
#reading data
df=pd.read_csv('/content/drive/MyDrive/Project data/train_data_evaluation_part_2.csv')
df.head()

,Unnamed: 0,ID,Nationality,Age,DaysSinceCreation,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,DaysSinceLastStay,DaysSinceFirstStay,DistributionChannel,MarketSegment,SRHighFloor,SRLowFloor,SRAccessibleRoom,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRNearElevator,SRAwayFromElevator,SRNoAlcoholInMiniBar,SRQuietRoom
0,0,1,PRT,51.0,150,45,371.0,105.3,1,0,3,8,5,151,1074,Corporate,Corporate,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2,PRT,NaN,1095,61,280.0,53.0,0,0,1,10,5,1100,1100,Travel Agent/Operator,Travel Agent/Operator,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,3,DEU,31.0,1095,0,0.0,0.0,0,0,0,0,0,-1,-1,Travel Agent/Operator,Travel Agent/Operator,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,4,FRA,60.0,1095,93,240.0,60.0,0,0,1,10,5,1100,1100,Travel Agent/Operator,Travel Agent/Operator,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,5,FRA,51.0,1095,0,0.0,0.0,0,0,0,0,0,-1,-1,Travel Agent/Operator,Travel Agent/Operator,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.isnull().sum()

Unnamed: 0                 0
ID                         0
Nationality                0
Age                     3746
DaysSinceCreation          0
AverageLeadTime            0
LodgingRevenue             0
OtherRevenue               0
BookingsCanceled           0
BookingsNoShowed           0
BookingsCheckedIn          0
PersonsNights              0
RoomNights                 0
DaysSinceLastStay          0
DaysSinceFirstStay         0
DistributionChannel        0
MarketSegment              0
SRHighFloor                0
SRLowFloor                 0
SRAccessibleRoom           0
SRMediumFloor              0
SRBathtub                  0
SRShower                   0
SRCrib                     0
SRKingSizeBed              0
SRTwinBed                  0
SRNearElevator             0
SRAwayFromElevator         0
SRNoAlcoholInMiniBar       0
SRQuietRoom                0
dtype: int64

In [ ]:
#converting checkedin column to either 0 /1
def convert(x):
  if(x==0):
    return 0   #not checked in
  else:         #checked in
    return 1

In [ ]:
#preprocessing
def preprocessing1(df):
  cols = ['Nationality']
  x_temp = df.groupby(cols)['Age'].median().reset_index()
  df['Age'] = df['Age'].fillna(df[cols].reset_index().merge(x_temp, how='left', on=cols).set_index('index')['Age'])
  df['CheckedIn/Not']=df['BookingsCheckedIn'].apply(lambda x:convert(x))
  df.drop('BookingsCheckedIn',axis=1,inplace=True)
  df.drop(['Unnamed: 0','ID','DaysSinceCreation','RoomNights','DaysSinceLastStay','Nationality'],axis=1,inplace=True)
  df['Total Revenue']=df['LodgingRevenue']+df['OtherRevenue']
  df.drop(['LodgingRevenue','OtherRevenue'],axis=1,inplace=True)
  df.drop(df[(df['Age']<1)|(df['Age'] >=120)&(df['Age'] <=150)].index,inplace=True)
  df.drop(df[df['AverageLeadTime']<0].index,inplace=True)
  df.drop_duplicates(inplace=True)
  df.drop(['BookingsNoShowed','BookingsCanceled','SRNoAlcoholInMiniBar','SRNearElevator'],axis=1,inplace=True)
  return df



In [ ]:
df_mod=preprocessing1(df)

In [ ]:
mk = df[["MarketSegment"]]

mk = pd.get_dummies(mk, drop_first= True)

mk.head()

,MarketSegment_Complementary,MarketSegment_Corporate,MarketSegment_Direct,MarketSegment_Groups,MarketSegment_Other,MarketSegment_Travel Agent/Operator
0,0,1,0,0,0,0
1,0,0,0,0,0,1
2,0,0,0,0,0,1
3,0,0,0,0,0,1
4,0,0,0,0,0,1


In [ ]:

dc = df[["DistributionChannel"]]

dc = pd.get_dummies(dc, drop_first= True)

dc.head()

,DistributionChannel_Direct,DistributionChannel_Electronic Distribution,DistributionChannel_Travel Agent/Operator
0,0,0,0
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1


In [ ]:
df_mod.drop(['DistributionChannel','MarketSegment'],axis=1,inplace=True)

In [ ]:

df_mod = pd.concat([df_mod, mk, dc], axis = 1)

In [ ]:
df_mod.isnull().sum()

Age                                            0
AverageLeadTime                                0
PersonsNights                                  0
DaysSinceFirstStay                             0
SRHighFloor                                    0
SRLowFloor                                     0
SRAccessibleRoom                               0
SRMediumFloor                                  0
SRBathtub                                      0
SRShower                                       0
SRCrib                                         0
SRKingSizeBed                                  0
SRTwinBed                                      0
SRAwayFromElevator                             0
SRQuietRoom                                    0
CheckedIn/Not                                  0
Total Revenue                                  0
MarketSegment_Complementary                    0
MarketSegment_Corporate                        0
MarketSegment_Direct                           0
MarketSegment_Groups

In [ ]:
df_mod.describe()

,Age,AverageLeadTime,PersonsNights,DaysSinceFirstStay,SRHighFloor,SRLowFloor,SRAccessibleRoom,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRAwayFromElevator,SRQuietRoom,CheckedIn/Not,Total Revenue,MarketSegment_Complementary,MarketSegment_Corporate,MarketSegment_Direct,MarketSegment_Groups,MarketSegment_Other,MarketSegment_Travel Agent/Operator,DistributionChannel_Direct,DistributionChannel_Electronic Distribution,DistributionChannel_Travel Agent/Operator
count,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000,64154.000000
mean,46.834601,82.056099,5.916576,512.564330,0.051501,0.001808,0.000327,0.001138,0.003538,0.002229,0.012532,0.352729,0.137731,0.004520,0.093042,0.961670,467.030418,0.007139,0.031456,0.135097,0.113711,0.566013,0.142797,0.141410,0.007872,0.812825
std,14.953652,89.017293,4.399195,312.146798,0.221019,0.042484,0.018090,0.033714,0.059379,0.047160,0.111245,0.477823,0.344620,0.067082,0.290493,0.191992,455.267088,0.084192,0.174547,0.341830,0.317462,0.495627,0.349868,0.348446,0.088373,0.390055
min,1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,36.000000,12.000000,3.000000,232.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,219.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,46.000000,53.000000,6.000000,515.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,368.850000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
75%,57.000000,125.000000,8.000000,793.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,570.500000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000
max,114.000000,588.000000,116.000000,1186.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,23365.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


1. .Age is having negative values and .So need to remove 

2. AverageLeadTime has negative value and is left squed.Need to be handled
3. BookingsNoShowed,BookingsCanceled,SRNoAlcoholInMiniBar,SRNearElevator	has 0 standard Deviation.Drop those
4. BookingsCanceled has 82462 0 values.(99.4%)
5. BookingsNoShowed has 82536 0 values(99.9%)




In [ ]:
%matplotlib notebook
fig_dims = (6, 4)
fig, ax = plt.subplots(figsize=fig_dims)
sns.barplot(x = "DistributionChannel", y = "Age", ax=ax, data=df)


<IPython.core.display.Javascript object>

In [ ]:
df.columns

Index(['Unnamed: 0', 'ID', 'Nationality', 'Age', 'DaysSinceCreation',
       'AverageLeadTime', 'LodgingRevenue', 'OtherRevenue', 'BookingsCanceled',
       'BookingsNoShowed', 'BookingsCheckedIn', 'PersonsNights', 'RoomNights',
       'DaysSinceLastStay', 'DaysSinceFirstStay', 'DistributionChannel',
       'MarketSegment', 'SRHighFloor', 'SRLowFloor', 'SRAccessibleRoom',
       'SRMediumFloor', 'SRBathtub', 'SRShower', 'SRCrib', 'SRKingSizeBed',
       'SRTwinBed', 'SRNearElevator', 'SRAwayFromElevator',
       'SRNoAlcoholInMiniBar', 'SRQuietRoom'],
      dtype='object')

In [ ]:
len(df['ID'].unique())

82580

In [ ]:

#! pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     / 21.9 MB 50 kB/s
     |████████████████████████████████| 102 kB 32.4 MB/s 
     |████████████████████████████████| 690 kB 56.4 MB/s 
     |████████████████████████████████| 4.7 MB 50.7 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
     |████████████████████████████████| 296 kB 57.2 MB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.2.0-py2.py3-none-any.whl size=261257 sha256=fb254276a235ee5201d7dcd234bcafe823835b59a2e8678667d41cc32dacb8df
  Stored in directory: /tmp/pip-ephem-wheel-cache-7zr2r2ns/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=b42ad43ae300ccbfb272b5e9a28db15aaf568d70fdda6ec42f8ecc279189aad1
  Stored in directory: /root/.cache/pip/wheels/70/e1/52/5b14d250ba868768823940c3229e9950d201a26d0bd3ee8655
Successfully built pandas-pro

In [ ]:
#profile = ProfileReport(df, title="data set", html={'style' : {'full_width':True}})
#profile.to_file(output_file="name of html file.html") 

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:4812: RuntimeWarning: overflow encountered in long_scalars
  (2 * xtie * ytie) / m + x0 * y0 / (9 * m * (size - 2)))
/usr/local/lib/python3.7/dist-packages/scipy/stats/stats.py:4814: RuntimeWarning: invalid value encountered in sqrt
  np.sqrt(var) / np.sqrt(2)))


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82522 entries, 0 to 82579
Data columns (total 25 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           82522 non-null  int64  
 1   ID                   82522 non-null  int64  
 2   Nationality          82522 non-null  object 
 3   Age                  82522 non-null  float64
 4   DaysSinceCreation    82522 non-null  int64  
 5   AverageLeadTime      82522 non-null  int64  
 6   PersonsNights        82522 non-null  int64  
 7   RoomNights           82522 non-null  int64  
 8   DaysSinceLastStay    82522 non-null  int64  
 9   DaysSinceFirstStay   82522 non-null  int64  
 10  DistributionChannel  82522 non-null  object 
 11  MarketSegment        82522 non-null  object 
 12  SRHighFloor          82522 non-null  int64  
 13  SRLowFloor           82522 non-null  int64  
 14  SRAccessibleRoom     82522 non-null  int64  
 15  SRMediumFloor        82522 non-null 

In [ ]:
df.Nationality.value_counts().to_dict()

{'FRA': 12298,
 'PRT': 11375,
 'DEU': 10156,
 'GBR': 8603,
 'ESP': 4863,
 'USA': 3397,
 'ITA': 3298,
 'BEL': 3110,
 'BRA': 2705,
 'NLD': 2696,
 'CHE': 2079,
 'IRL': 1991,
 'CAN': 1513,
 'AUT': 1486,
 'SWE': 1221,
 'ISR': 896,
 'CHN': 873,
 'NOR': 789,
 'POL': 754,
 'AUS': 722,
 'DNK': 666,
 'FIN': 632,
 'RUS': 578,
 'ROU': 475,
 'HUN': 327,
 'JPN': 253,
 'CZE': 251,
 'GRC': 238,
 'LUX': 224,
 'IND': 193,
 'ARG': 179,
 'KOR': 178,
 'MEX': 177,
 'AGO': 177,
 'MAR': 167,
 'TUR': 163,
 'BGR': 161,
 'SRB': 147,
 'HRV': 133,
 'EST': 126,
 'NZL': 96,
 'UKR': 94,
 'LVA': 91,
 'CHL': 77,
 'MOZ': 76,
 'IRN': 76,
 'SVK': 75,
 'DZA': 75,
 'TWN': 70,
 'COL': 69,
 'ZAF': 64,
 'LTU': 63,
 'SVN': 60,
 'CYP': 57,
 'ISL': 54,
 'AZE': 52,
 'PHL': 49,
 'THA': 39,
 'PER': 38,
 'LBN': 37,
 'SGP': 36,
 'URY': 33,
 'TUN': 32,
 'MLT': 32,
 'BLR': 30,
 'SAU': 30,
 'ARE': 29,
 'VNM': 28,
 'VEN': 27,
 'CMR': 25,
 'EGY': 24,
 'BIH': 22,
 'ECU': 21,
 'IDN': 21,
 'MYS': 20,
 'MKD': 18,
 'KAZ': 17,
 'PAN': 17,
 'JOR'

In [ ]:
df.Nationality=df.Nationality.map(df.Nationality.value_counts().to_dict())

In [ ]:
df_mod.head()

,Age,AverageLeadTime,PersonsNights,DaysSinceFirstStay,SRHighFloor,SRLowFloor,SRAccessibleRoom,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRAwayFromElevator,SRQuietRoom,CheckedIn/Not,Total Revenue,MarketSegment_Complementary,MarketSegment_Corporate,MarketSegment_Direct,MarketSegment_Groups,MarketSegment_Other,MarketSegment_Travel Agent/Operator,DistributionChannel_Direct,DistributionChannel_Electronic Distribution,DistributionChannel_Travel Agent/Operator
0,51.000000,45,8,1074,0,0,0,0,0,0,0,0,0,0,0,1,476.3,0,1,0,0,0,0,0,0,0
1,44.038099,61,10,1100,0,0,0,0,0,0,0,0,0,0,0,1,333.0,0,0,0,0,0,1,0,0,1
2,31.000000,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,1,0,0,1
3,60.000000,93,10,1100,0,0,0,0,0,0,0,0,0,0,0,1,300.0,0,0,0,0,0,1,0,0,1
4,51.000000,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,1,0,0,1


In [ ]:
Y=df_mod['CheckedIn/Not']
X=df_mod.drop(['CheckedIn/Not','MarketSegment_Groups'],axis=1)

NameError: ignored

In [ ]:
Y.value_counts()

1    61695
0     2459
Name: CheckedIn/Not, dtype: int64

In [ ]:
#balancing data
from imblearn.over_sampling import SVMSMOTE
sm = SVMSMOTE(random_state=42)
x_res,y_res=sm.fit_resample(X,Y)

In [ ]:
y_res.value_counts()

1    61695
0    61695
Name: CheckedIn/Not, dtype: int64

In [ ]:

import matplotlib.pyplot  as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import tensorflow as tf
tf.random.set_seed(3)      
from tensorflow import keras

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(x_res,y_res,test_size=0.2,random_state=42)

In [ ]:
#data scaling
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
x_train_std=scaler.fit_transform(X_train)
x_test_std=scaler.transform(X_test)

In [ ]:
x_train_std=pd.DataFrame(x_train_std)
x_test_std=pd.DataFrame(x_test_std)

In [ ]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66017 entries, 28852 to 15795
Data columns (total 27 columns):
 #   Column                                     Non-Null Count  Dtype 
---  ------                                     --------------  ----- 
 0   onehotencoder__x0_Corporate                66017 non-null  object
 1   onehotencoder__x0_Direct                   66017 non-null  object
 2   onehotencoder__x0_Electronic Distribution  66017 non-null  object
 3   onehotencoder__x0_Travel Agent/Operator    66017 non-null  object
 4   onehotencoder__x1_Aviation                 66017 non-null  object
 5   onehotencoder__x1_Complementary            66017 non-null  object
 6   onehotencoder__x1_Corporate                66017 non-null  object
 7   onehotencoder__x1_Direct                   66017 non-null  object
 8   onehotencoder__x1_Groups                   66017 non-null  object
 9   onehotencoder__x1_Other                    66017 non-null  object
 10  onehotencoder__x1_Travel Agent

In [ ]:
#layer initialisation for NN
LAYERS= [
    tf.keras.layers.Flatten(input_shape=(X.shape[1],)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(500,use_bias=False,kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(350, use_bias=False,kernel_initializer=tf.keras.initializers.HeNormal()),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation("relu"),
    tf.keras.layers.Dense(2, activation="sigmoid")
]

In [ ]:
tenserboard_cb=tf.keras.callbacks.EarlyStopping(patience=5,restore_best_weights=True)

In [ ]:
model = tf.keras.models.Sequential(LAYERS)

In [ ]:
#compile NN
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(lr=1e-3),
              metrics=["accuracy"])

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 24)                0         
                                                                 
 batch_normalization_3 (Batc  (None, 24)               96        
 hNormalization)                                                 
                                                                 
 dense_3 (Dense)             (None, 500)               12000     
                                                                 
 batch_normalization_4 (Batc  (None, 500)              2000      
 hNormalization)                                                 
                                                                 
 activation_2 (Activation)   (None, 500)               0         
                                                                 
 dense_4 (Dense)             (None, 350)              

In [ ]:
#Training
history=model.fit(x_train_std,Y_train,validation_split=0.1,epochs=20,callbacks=tenserboard_cb)

Epoch 1/20
2301/2301 [==============================] - 9s 4ms/step - loss: 0.1337 - accuracy: 0.9620 - val_loss: 0.0691 - val_accuracy: 0.9861
Epoch 2/20
2301/2301 [==============================] - 9s 4ms/step - loss: 0.0564 - accuracy: 0.9856 - val_loss: 0.0435 - val_accuracy: 0.9903
Epoch 3/20
2301/2301 [==============================] - 9s 4ms/step - loss: 0.0454 - accuracy: 0.9884 - val_loss: 0.0528 - val_accuracy: 0.9902
Epoch 4/20
2301/2301 [==============================] - 9s 4ms/step - loss: 0.0388 - accuracy: 0.9896 - val_loss: 0.0280 - val_accuracy: 0.9921
Epoch 5/20
2301/2301 [==============================] - 9s 4ms/step - loss: 0.0377 - accuracy: 0.9890 - val_loss: 0.0268 - val_accuracy: 0.9921
Epoch 6/20
2301/2301 [==============================] - 9s 4ms/step - loss: 0.0328 - accuracy: 0.9905 - val_loss: 0.0207 - val_accuracy: 0.9936
Epoch 7/20
2301/2301 [==============================] - 8s 4ms/step - loss: 0.0328 - accuracy: 0.9909 - val_loss: 0.0203 - val_accuracy:

In [ ]:
loss,accuracy=model.evaluate(x_test_std,Y_test)

640/640 [==============================] - 1s 2ms/step - loss: 0.0229 - accuracy: 0.9951


In [ ]:
y_pred=model.predict(x_test_std)
y_pred

array([[0.9661687 , 0.02811748],
       [0.00242998, 0.99538714],
       [0.93876123, 0.04627861],
       ...,
       [0.02572052, 0.9576319 ],
       [0.00177417, 0.99677974],
       [0.10482612, 0.9997911 ]], dtype=float32)

In [ ]:
#converting prediction proba to class labels.
#argmax function gives position of max value in lsit
y_pred_labels=[np.argmax(i) for i in y_pred]
print(y_pred_labels)


[0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 

In [ ]:
confusion_matrix(Y_test,y_pred_labels)

array([[ 7702,     8],
       [   92, 12647]])

In [ ]:
#moderately FN rates

df_test=pd.read_csv('/content/drive/MyDrive/Project data/test_data_evaluation_part2.csv')
df_test.head()

,Unnamed: 0,ID,Nationality,Age,DaysSinceCreation,AverageLeadTime,LodgingRevenue,OtherRevenue,BookingsCanceled,BookingsNoShowed,BookingsCheckedIn,PersonsNights,RoomNights,DaysSinceLastStay,DaysSinceFirstStay,DistributionChannel,MarketSegment,SRHighFloor,SRLowFloor,SRAccessibleRoom,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRNearElevator,SRAwayFromElevator,SRNoAlcoholInMiniBar,SRQuietRoom
0,82590,82591,SGP,47.0,11,0,0.00,0.0,0,0,0,0,0,-1,-1,Travel Agent/Operator,Other,0,0,0,0,0,0,0,0,0,0,0,0,0
1,82591,82592,SGP,16.0,11,130,483.48,155.1,0,0,1,15,5,16,16,Travel Agent/Operator,Other,0,0,0,0,0,0,0,0,0,0,0,0,0
2,82592,82593,SGP,15.0,11,0,0.00,0.0,0,0,0,0,0,-1,-1,Travel Agent/Operator,Other,0,0,0,0,0,0,0,0,0,0,0,0,0
3,82593,82594,SGP,12.0,11,0,0.00,0.0,0,0,0,0,0,-1,-1,Travel Agent/Operator,Other,0,0,0,0,0,0,0,0,0,0,0,0,0
4,82594,82595,PRT,NaN,11,0,0.00,0.0,0,0,0,0,0,-1,-1,Travel Agent/Operator,Other,1,0,0,0,0,0,0,1,0,0,0,0,0


In [ ]:
df_test['MarketSegment'].value_counts()

Other                    578
Travel Agent/Operator    184
Direct                   177
Corporate                 34
Complementary             26
Aviation                   1
Name: MarketSegment, dtype: int64

In [ ]:
temp=df_test['ID']

In [ ]:
mod=preprocessing1(df_test)

In [ ]:
mk = mod[["MarketSegment"]]

mk = pd.get_dummies(mk, drop_first= True)

mk.head()

,MarketSegment_Complementary,MarketSegment_Corporate,MarketSegment_Direct,MarketSegment_Other,MarketSegment_Travel Agent/Operator
0,0,0,0,1,0
1,0,0,0,1,0
2,0,0,0,1,0
3,0,0,0,1,0
4,0,0,0,1,0


In [ ]:
dc= mod[["DistributionChannel"]]

dc = pd.get_dummies(dc, drop_first= True)

dc.head()

,DistributionChannel_Direct,DistributionChannel_Electronic Distribution,DistributionChannel_Travel Agent/Operator
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1


In [ ]:

df_mod = pd.concat([mod, mk, dc], axis = 1)
df_mod.drop(['DistributionChannel','MarketSegment'],axis=1,inplace=True)

In [ ]:
df_mod

,Age,AverageLeadTime,PersonsNights,DaysSinceFirstStay,SRHighFloor,SRLowFloor,SRAccessibleRoom,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRAwayFromElevator,SRQuietRoom,CheckedIn/Not,Total Revenue,MarketSegment_Complementary,MarketSegment_Corporate,MarketSegment_Direct,MarketSegment_Groups,MarketSegment_Other,MarketSegment_Travel Agent/Operator,DistributionChannel_Direct,DistributionChannel_Electronic Distribution,DistributionChannel_Travel Agent/Operator
0,51.0,45,8,1074,0,0,0,0,0,0,0,0,0,0,0,1,476.30,0,1,0,0,0,0,0,0,0
1,44.0,61,10,1100,0,0,0,0,0,0,0,0,0,0,0,1,333.00,0,0,0,0,0,1,0,0,1
2,31.0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,1,0,0,1
3,60.0,93,10,1100,0,0,0,0,0,0,0,0,0,0,0,1,300.00,0,0,0,0,0,1,0,0,1
4,51.0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82572,32.0,88,4,14,0,0,0,0,0,0,0,0,0,0,0,1,154.00,0,0,0,0,0,1,0,0,1
82574,41.0,7,1,13,0,0,0,0,0,0,0,0,0,0,0,1,82.18,0,0,0,0,1,0,0,0,1
82575,51.0,114,6,15,0,0,0,0,0,0,0,1,0,0,0,1,196.08,0,0,0,0,1,0,0,0,1
82577,50.0,18,3,15,0,0,0,0,0,0,0,0,0,0,0,1,477.00,0,0,0,0,1,0,0,0,1


In [ ]:
X1_scaled=scaler.transform(X1)

In [ ]:
s=X1_scaled.reshape(1,-1)

In [ ]:
X1=df_mod.drop('CheckedIn/Not',axis=1)
Y1=df_mod['CheckedIn/Not']

In [ ]:
X1.shape

(815, 24)

In [ ]:
X1=np.asarray(X1)

In [ ]:
X1

,Age,AverageLeadTime,PersonsNights,DaysSinceFirstStay,SRHighFloor,SRLowFloor,SRAccessibleRoom,SRMediumFloor,SRBathtub,SRShower,SRCrib,SRKingSizeBed,SRTwinBed,SRAwayFromElevator,SRQuietRoom,Total Revenue,MarketSegment_Complementary,MarketSegment_Corporate,MarketSegment_Direct,MarketSegment_Other,MarketSegment_Travel Agent/Operator,DistributionChannel_Direct,DistributionChannel_Electronic Distribution,DistributionChannel_Travel Agent/Operator
0,47.0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,1,0,0,0,1
1,16.0,130,15,16,0,0,0,0,0,0,0,0,0,0,0,638.58,0,0,0,1,0,0,0,1
2,15.0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,1,0,0,0,1
3,12.0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,1,0,0,0,1
4,40.0,0,0,-1,1,0,0,0,0,0,0,1,0,0,0,0.00,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
991,48.0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0.00,0,0,0,1,0,0,0,1
992,49.0,91,2,1,0,0,0,0,0,0,0,1,0,0,0,145.00,0,0,0,1,0,0,0,1
994,52.0,116,8,4,0,0,0,0,0,0,0,0,0,0,0,442.40,0,0,0,1,0,0,0,1
996,47.0,109,6,3,0,0,0,0,0,0,0,1,0,0,0,604.60,0,0,0,1,0,0,0,1


In [ ]:
scaled=X1_scaled.reshape(1,-1)



In [ ]:
input_data_std=scaler.transform(X1)
input_data_nparray=np.asarray(input_data_std)
#reshape data
input_data_reshaped=input_data_nparray.reshape(1,-1)
#standardize data
prediction=model.predict(input_data_reshaped)
print(prediction)
prediction_label=[np.argmax(prediction)]
print(prediction_label)

if(prediction_label[0]==0):
  print('wont  Arrived')
else:
  print('Will arrive')

[[7.9802787e-01 2.7815494e-01]
 [3.2675648e-04 9.9980301e-01]
 [8.3200687e-01 1.2863833e-01]
 ...
 [8.4875816e-01 2.3136020e-01]
 [1.3103236e-03 9.9901915e-01]
 [8.1281221e-01 9.8764271e-02]]
[125]
Will arrive


In [ ]:
prediction_label

[125]

In [ ]:
import pickle
# open a file, where you ant to store the data
file = open('model.pkl', 'wb')

# dump information to that file
pickle.dump(model, file)

In [ ]:
file1=open('encoder.pkl','wb')
pickle.dump()

In [ ]:
#machine learning models
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier

In [ ]:
svc = SVC(kernel='poly', gamma=1.0,degree=3,max_iter=50)
knc = KNeighborsClassifier()
mnb = MultinomialNB()
dtc = DecisionTreeClassifier(max_depth=5)
lrc = LogisticRegression(solver='liblinear', penalty='l1')
rfc = RandomForestClassifier(n_estimators=50, random_state=2)
abc = AdaBoostClassifier(n_estimators=50, random_state=2)
bc = BaggingClassifier(n_estimators=50, random_state=2)
etc = ExtraTreesClassifier(n_estimators=50, random_state=2)
gbdt = GradientBoostingClassifier(n_estimators=50,random_state=2)
xgb = XGBClassifier(n_estimators=50,random_state=2)
from sklearn.metrics import accuracy_score,precision_score

In [ ]:
clfs = {
    'SVC' : svc,
    'KN' : knc,  
    'DT': dtc, 
    'LR': lrc, 
    'RF': rfc, 
    'AdaBoost': abc, 
    'BgC': bc, 
    'ETC': etc,
    'GBDT':gbdt,
    'xgb':xgb
}

In [ ]:
def train_classifier(clf,X_train,y_train,X_test,y_test):
    clf.fit(X_train,y_train)
    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test,y_pred)
    precision = precision_score(y_test,y_pred)
    
    return accuracy,precision

In [ ]:
accuracy_scores = []
precision_scores = []

for name,clf in clfs.items():
    
    current_accuracy,current_precision = train_classifier(clf, x_train_std,Y_train,x_test_std,Y_test)
    
    print("For ",name)
    print("Accuracy - ",current_accuracy)
    print("Precision - ",current_precision)
    
    accuracy_scores.append(current_accuracy)
    precision_scores.append(current_precision)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=50).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


For  SVC
Accuracy -  0.9409789131751363
Precision -  0.9213908908537038
For  KN
Accuracy -  0.9971370368331223
Precision -  0.998387486898331
For  DT
Accuracy -  1.0
Precision -  1.0


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


For  LR
Accuracy -  0.9992292022243021
Precision -  1.0
For  RF
Accuracy -  1.0
Precision -  1.0
For  AdaBoost
Accuracy -  1.0
Precision -  1.0
For  BgC
Accuracy -  1.0
Precision -  1.0
For  ETC
Accuracy -  1.0
Precision -  1.0
For  GBDT
Accuracy -  1.0
Precision -  1.0
For  xgb
Accuracy -  1.0
Precision -  1.0


In [ ]:
import pickle
# open a file, where you ant to store the data
#file = open('svmmodel.pkl', 'wb')

# dump information to that file
#pickle.dump(svc, file)
pickle.dump(scaler,open('scaler.pkl','wb'))

In [ ]:
svc = SVC(kernel='poly', gamma=1.0,degree=1,max_iter=50)
svc.fit(X_train,Y_train)
#y_pred = SVC.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:289: ConvergenceWarning: Solver terminated early (max_iter=50).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  ConvergenceWarning,


SVC(degree=1, gamma=1.0, kernel='poly', max_iter=50)

In [ ]:
scaler_model=pickle.load((open('scaler.pkl','rb')))